In [2]:
import numpy as np
import pandas as pd

In [2]:
countries={ 'Austria': 'AT', 'Belgium': 'BE',  'Bulgaria': 'BG', 'Switzerland': 'CH', 'Czech Republic': 'CZ', 'Croatia': 'HR',  'Germany': 'DE', 'Denmark': 'DK', 'Estonia': 'EE', 'Spain': 'ES', 'Finland': 'FI', 'France': 'FR',  'Greece': 'GR', 'Hungary': 'HU', 'Ireland': 'IE', 'Italy': 'IT', 'Lithuania': 'LT', 'Latvia': 'LV', 'Montenegro': 'ME','Netherlands': 'NL', 'Norway': 'NO', 'Poland': 'PL', 'Portugal': 'PT', 'Serbia': 'RS', 'Sweden': 'SE', 'Slovenia': 'SI', 'Slovakia': 'SK', 'United Kingdom': 'UK'}

abbr_list=list(countries.values())

In [2]:
fuels=['Biomass', 'Fossil Brown coal/Lignite', 'Fossil Coal-derived gas', 'Fossil Gas', 'Fossil Hard coal', 'Fossil Oil', 'Fossil Oil shale', 'Fossil Peat', 'Geothermal', 'Hydro Pumped Storage', 'Hydro Pumped Storage', 'Hydro Run-of-river and poundage', 'Hydro Water Reservoir', 'Marine', 'Nuclear', 'Other', 'Other renewable', 'Solar', 'Waste', 'Wind Offshore', 'Wind Onshore']

In [19]:
df=pd.read_csv('../Data Sources/ENTSO-E/2018/Generation/Germany.csv')

In [63]:
df = pd.read_csv('../Data Sources/ENTSO-E/Load/Germany.csv')
df


,Time (CET),Day-ahead Total Load Forecast [MW] - Germany (DE),Actual Total Load [MW] - Germany (DE)
0,01.01.2019 00:00 - 01.01.2019 00:15,43769.0,43671.0
1,01.01.2019 00:15 - 01.01.2019 00:30,43572.0,43195.0
2,01.01.2019 00:30 - 01.01.2019 00:45,43025.0,42704.0
3,01.01.2019 00:45 - 01.01.2019 01:00,42487.0,42481.0
4,01.01.2019 01:00 - 01.01.2019 01:15,41978.0,42139.0
...,...,...,...
35039,31.12.2019 22:45 - 31.12.2019 23:00,49636.0,45675.0
35040,31.12.2019 23:00 - 31.12.2019 23:15,48812.0,45108.0
35041,31.12.2019 23:15 - 31.12.2019 23:30,48087.0,44751.0
35042,31.12.2019 23:30 - 31.12.2019 23:45,47442.0,44355.0


In [21]:
df['Total'] = df.iloc[:,2:].sum(axis=1)
df

,Area,MTU,Biomass - Actual Aggregated [MW],Fossil Brown coal/Lignite - Actual Aggregated [MW],Fossil Coal-derived gas - Actual Aggregated [MW],Fossil Gas - Actual Aggregated [MW],Fossil Hard coal - Actual Aggregated [MW],Fossil Oil - Actual Aggregated [MW],Fossil Oil shale - Actual Aggregated [MW],Fossil Peat - Actual Aggregated [MW],...,Hydro Water Reservoir - Actual Aggregated [MW],Marine - Actual Aggregated [MW],Nuclear - Actual Aggregated [MW],Other - Actual Aggregated [MW],Other renewable - Actual Aggregated [MW],Solar - Actual Aggregated [MW],Waste - Actual Aggregated [MW],Wind Offshore - Actual Aggregated [MW],Wind Onshore - Actual Aggregated [MW],Total
0,Germany (DE),01.01.2018 00:00 - 01.01.2018 00:15 (CET),4779.0,6671.0,NaN,2408.0,1759.0,187.0,n/e,n/e,...,44.0,n/e,6345.0,329.0,144.0,0.0,370.0,2973.0,27982.0,57312.0
1,Germany (DE),01.01.2018 00:15 - 01.01.2018 00:30 (CET),4778.0,6712.0,NaN,2411.0,1724.0,187.0,n/e,n/e,...,45.0,n/e,5956.0,329.0,144.0,0.0,365.0,2991.0,28618.0,57597.0
2,Germany (DE),01.01.2018 00:30 - 01.01.2018 00:45 (CET),4778.0,6739.0,NaN,2411.0,1687.0,187.0,n/e,n/e,...,42.0,n/e,5753.0,329.0,144.0,0.0,370.0,2921.0,28953.0,57626.0
3,Germany (DE),01.01.2018 00:45 - 01.01.2018 01:00 (CET),4782.0,6773.0,NaN,2395.0,1674.0,187.0,n/e,n/e,...,48.0,n/e,5361.0,329.0,144.0,0.0,370.0,2785.0,29088.0,57505.0
4,Germany (DE),01.01.2018 01:00 - 01.01.2018 01:15 (CET),4768.0,6828.0,NaN,2387.0,1609.0,188.0,n/e,n/e,...,47.0,n/e,4942.0,329.0,144.0,0.0,370.0,2895.0,29643.0,57993.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35039,Germany (DE),31.12.2018 22:45 - 31.12.2018 23:00 (CET),4963.0,9492.0,NaN,3951.0,3570.0,470.0,n/e,n/e,...,28.0,n/e,8972.0,383.0,117.0,0.0,496.0,3356.0,17289.0,55243.0
35040,Germany (DE),31.12.2018 23:00 - 31.12.2018 23:15 (CET),4949.0,9159.0,NaN,3755.0,3627.0,470.0,n/e,n/e,...,52.0,n/e,9002.0,384.0,117.0,0.0,501.0,3348.0,17728.0,55180.0
35041,Germany (DE),31.12.2018 23:15 - 31.12.2018 23:30 (CET),4964.0,8963.0,NaN,3745.0,3627.0,470.0,n/e,n/e,...,53.0,n/e,9010.0,384.0,117.0,0.0,500.0,3046.0,18120.0,55090.0
35042,Germany (DE),31.12.2018 23:30 - 31.12.2018 23:45 (CET),4925.0,8615.0,NaN,3734.0,3643.0,469.0,n/e,n/e,...,77.0,n/e,9050.0,384.0,117.0,0.0,500.0,2883.0,18610.0,55442.0


In [22]:
df=pd.read_csv('temp.csv')
df

,a,b,c,d
0,1,5,d,3
1,2,4,c,4
2,3,3,h,5
3,4,2,t,6
4,5,1,b,7


In [23]:
df = df*2
df

,a,b,c,d
0,2,10,dd,6
1,4,8,cc,8
2,6,6,hh,10
3,8,4,tt,12
4,10,2,bb,14
